In [95]:
import pathlib

BASE_DIR = pathlib.Path().resolve().parent
DATASET_DIR = BASE_DIR / "datasets"
EXPORT_DIR = DATASET_DIR / "exports"
EXPORT_DIR.mkdir(exist_ok=True, parents=True)
SPAM_DATASET_PATH = EXPORT_DIR / "spam-dataset.csv"

ZIPS_DIR = DATASET_DIR / 'zips'
ZIPS_DIR.mkdir(exist_ok=True, parents=True)

SPAM_SMS_ZIP_PATH = ZIPS_DIR / "sms-spam-dataset.zip"
SPAM_YOUTUBE_ZIP_PATH = ZIPS_DIR / "youtube-spam-dataset.zip"

In [96]:
SMS_SPAM_ZIP = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
YOUTUBE_SPAM_ZIP = "https://archive.ics.uci.edu/static/public/380/youtube+spam+collection.zip"

In [97]:
!curl $SMS_SPAM_ZIP -o $SPAM_SMS_ZIP_PATH
!curl $YOUTUBE_SPAM_ZIP -o $SPAM_YOUTUBE_ZIP_PATH

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  8054    0  8054    0     0   4944      0 --:--:--  0:00:01 --:--:--  4950
100 40822    0 40822    0     0  15342      0 --:--:--  0:00:02 --:--:-- 15352
100  123k    0  123k    0     0  33782      0 --:--:--  0:00:03 --:--:-- 33794
100  179k    0  179k    0     0  38604      0 --:--:--  0:00:04 --:--:-- 38616
100  198k    0  198k    0     0  40812      0 --:--:--  0:00:04 --:--:-- 47806
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:-

In [98]:
SPAM_CLASSIFIER_DIR = DATASET_DIR / "spam-classifier"
SMS_SPAM_DIR = SPAM_CLASSIFIER_DIR / "spam-sms"
YOUTUBE_SPAM_DIR = SPAM_CLASSIFIER_DIR / "youtube-spam"

SMS_SPAM_DIR.mkdir(exist_ok=True, parents=True)
YOUTUBE_SPAM_DIR.mkdir(exist_ok=True, parents=True)

In [99]:
!PowerShell Expand-Archive -Path $SPAM_SMS_ZIP_PATH -DestinationPath $SMS_SPAM_DIR
!PowerShell Expand-Archive -Path $SPAM_YOUTUBE_ZIP_PATH -DestinationPath $YOUTUBE_SPAM_DIR

. : File C:\Users\p-rat\OneDrive - Mahidol University\??????\WindowsPowerShell\profile.ps1 cannot be loaded because 
running scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:1 char:3
+ . 'C:\Users\p-rat\OneDrive - Mahidol University\??????\WindowsPowerSh ...
+   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess
ExpandArchiveHelper : Failed to create file 'C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets
\spam-classifier\spam-sms\SMSSpamCollection' while expanding the archive file 
'C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\zips\sms-spam-dataset.zip' contents as the 
file 'C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\spam-sms\SMSSpamCollec
tion' alrea

In [100]:
#Extract, Review & Combine Datasets

In [101]:
sms_spam_input_path = SMS_SPAM_DIR / "SMSSpamCollection"
#sms_spam_input_path.read_text()

In [102]:
for path in YOUTUBE_SPAM_DIR.glob("*"):
    print(path)

C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\youtube-spam\Youtube01-Psy.csv
C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\youtube-spam\Youtube02-KatyPerry.csv
C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\youtube-spam\Youtube03-LMFAO.csv
C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\youtube-spam\Youtube04-Eminem.csv
C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\youtube-spam\Youtube05-Shakira.csv
C:\study\Lecture_Workshop\MERN\MLOps_project\keras_fastapi\ai-api\datasets\spam-classifier\youtube-spam\__MACOSX


In [103]:
import pandas as pd

In [104]:
sms_spam_input_path = SMS_SPAM_DIR / "SMSSpamCollection"
sms_df = pd.read_csv(sms_spam_input_path, sep='\t', header=None)
sms_df.columns = ['label', 'text']
sms_df['source'] = 'sms-spam'
sms_df.tail()


,label,text,source
5567,spam,This is the 2nd time we have tried 2 contact u...,sms-spam
5568,ham,Will ü b going to esplanade fr home?,sms-spam
5569,ham,"Pity, * was in mood for that. So...any other s...",sms-spam
5570,ham,The guy did some bitching but I acted like i'd...,sms-spam
5571,ham,Rofl. Its true to its name,sms-spam


In [105]:
1 == True

True

In [106]:
my_dfs = []

for path in YOUTUBE_SPAM_DIR.glob("*.csv"):
    raw_df = pd.read_csv(path)
    raw_df.rename(columns={"CLASS": "raw_label", 'CONTENT': "text"}, inplace=True)
    raw_df['label'] = raw_df['raw_label'].apply(lambda x: "spam" if str(x) == '1' else "ham")
    raw_df['raw_source'] = str(path.name)
    raw_df['source'] = 'youtube-spam'
    df = raw_df.copy()[['label','text', 'source']]
    my_dfs.append(df)
    # print(df.head())

yt_df = pd.concat(my_dfs)

In [107]:
yt_df.head()

,label,text,source
0,spam,"Huh, anyway check out this you[tube] channel: ...",youtube-spam
1,spam,Hey guys check out my new channel and our firs...,youtube-spam
2,spam,just for test I have to say murdev.com,youtube-spam
3,spam,me shaking my sexy ass on my channel enjoy ^_^ ﻿,youtube-spam
4,spam,watch?v=vtaRGgvGtWQ Check this out .﻿,youtube-spam


In [108]:
df = pd.concat([sms_df, yt_df])
df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",sms-spam
1,ham,Ok lar... Joking wif u oni...,sms-spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,sms-spam
3,ham,U dun say so early hor... U c already then say...,sms-spam
4,ham,"Nah I don't think he goes to usf, he lives aro...",sms-spam


In [109]:
df.to_csv(SPAM_DATASET_PATH, index=False)